In [ ]:
import random
import io
import binvox_rw
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from tensorflow.keras.layers import Input, Conv2D, Conv3DTranspose, Concatenate, Dropout
from tensorflow.keras.layers import MaxPool2D, UpSampling3D, Reshape, Flatten, GlobalMaxPool2D
from tensorflow.keras.layers import BatchNormalization, Activation, Add, Dense, MaxPool3D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import Loss, BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.metrics import BinaryIoU
from tensorflow_addons.losses import GIoULoss
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def center(voxin):
    f = np.squeeze(np.max(voxin, axis=(2, 1)))
    s = np.squeeze(np.max(voxin, axis=(2, 0)))
    t = np.squeeze(np.max(voxin, axis=(1, 0)))

    nzf = np.sum(f == 0) 
    nzs = np.sum(s == 0)
    nzt = np.sum(t == 0)

    i_f = np.argmax(f)
    l_f = np.sum(f > 0)
    i_s = np.argmax(s)
    l_s = np.sum(s > 0)
    i_t = np.argmax(t)
    l_t = np.sum(t > 0)

    zfl = np.floor(nzf/2.0).astype(int)
    zfr = nzf - zfl
    zsl = np.floor(nzs/2.0).astype(int)
    zsr = nzs - zsl
    ztl = np.floor(nzt/2.0).astype(int)
    ztr = nzt - ztl  

    voxout = np.zeros_like(voxin)
    voxout[zfl:128-zfr, zsl:128-zsr, ztl:128-ztr] = voxin[i_f:i_f+l_f, i_s:i_s+l_s, i_t:i_t+l_t,]
    return voxout

In [ ]:
class voxel_gen(Sequence):
    def __init__(self, y_set, batch_size=32, dim_image=(512, 512, 3), dim_voxel=(128, 128, 128)):
        self.y = y_set
        self.batch_size = batch_size
        self.dim_voxel = dim_voxel
        self.dim_image = dim_image

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def __getitem__(self, index):
        y_set_temp = self.y[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(y_set_temp)
        return [X, Y]
    
    def __data_generation(self, y_set_temp):
        X0 = np.empty((self.batch_size, *self.dim_image))
        X1 = np.empty((self.batch_size, *self.dim_image))
        X2 = np.empty((self.batch_size, *self.dim_image))
        X3 = np.empty((self.batch_size, *self.dim_image))
        Y = np.empty((self.batch_size, *self.dim_voxel, 1))

        for i, stem in enumerate(y_set_temp):
            view0 = f"models/models-screenshots/view0/data/{stem}-00.png"
            view1 = f"models/models-screenshots/view1/data/{stem}-01.png"
            view2 = f"models/models-screenshots/view2/data/{stem}-02.png"
            view3 = f"models/models-screenshots/view3/data/{stem}-03.png"
            X0[i,] = np.array(Image.open(view0))[:, :, 0:3]
            X1[i,] = np.array(Image.open(view1))[:, :, 0:3]
            X2[i,] = np.array(Image.open(view2))[:, :, 0:3]
            X3[i,] = np.array(Image.open(view3))[:, :, 0:3]
            vox = f"models/models-binvox-solid/data/{stem}.binvox"
            with open(str(vox), 'rb') as f:
                model = binvox_rw.read_as_3d_array(f)
            Y[i,] = np.expand_dims(center(model.data), axis=-1) * 1
    
        return [[X0, X1, X2, X3], Y]
    
    def on_epoch_end(self):
        self.y = random.shuffle(self.y)

In [ ]:
def customGen(batch_size=32):
    voxels = Path("models/models-binvox-solid/data")
    fvoxels = [f.stem for f in voxels.iterdir() if f.is_file()]
    out = voxel_gen(fvoxels, batch_size)
    while True:
        for x, y in out:
            yield [x, y]

In [ ]:
def encode_decode():
    inview0 = tf.keras.applications.resnet_v2.preprocess_input(Input(shape=(512, 512, 3), name='view0'))
    inview1 = tf.keras.applications.resnet_v2.preprocess_input(Input(shape=(512, 512, 3), name='view1'))
    inview2 = tf.keras.applications.resnet_v2.preprocess_input(Input(shape=(512, 512, 3), name='view2'))
    inview3 = tf.keras.applications.resnet_v2.preprocess_input(Input(shape=(512, 512, 3), name='view3'))
   
    encoder = Sequential([
            Conv2D(32, 3, activation='relu', padding='same'),
            MaxPool2D((4, 4)),
            Conv2D(32, 5, activation='relu', padding='same'),
            MaxPool2D((4, 4)),
            Conv2D(64, 7, activation='relu', padding='same'),
            MaxPool2D((4, 4)),
            Conv2D(64, 5, activation='relu', padding='same'),
            MaxPool2D((2, 2)),
            Conv2D(128, 3, activation='relu', padding='same'),
            MaxPool2D((2, 2)),
            Conv2D(128, 3, activation='relu', padding='same'),
            MaxPool2D((2, 2)),
            Reshape((-1, 1)),
    ], name='encoder')
    
    x0 = encoder(inview0)
    x1 = encoder(inview1)
    x2 = encoder(inview2)
    x3 = encoder(inview3)
    
    combined = Concatenate(axis=-1)([x0, x1, x2, x3])
    
    FC = Sequential([
        Dropout(rate=0.5, noise_shape=[None, 1, 4]),
        Flatten(),
        Dense(256),
        Dense(1024),
        Reshape((2, 2, 2, 128))
    ], name='FC')(combined)
    
    decoder = Sequential([
            Conv3DTranspose(32, 5, activation='relu', padding='same'),
            UpSampling3D(4),
            Conv3DTranspose(16, 3, activation='relu', padding='same'),
            UpSampling3D(2),
            Conv3DTranspose(8, 3, activation='relu', padding='same'),
            UpSampling3D(2),
            Conv3DTranspose(4, 3, activation='relu', padding='same'),
            UpSampling3D(2),
            Conv3DTranspose(2, 3, activation='relu', padding='same'),
            UpSampling3D(2)
    ], name='decoder')(FC)
    
    out = Conv3DTranspose(1, 3, activation='sigmoid', padding='same')(decoder)
    
    return Model(inputs=[inview0, inview1, inview2, inview3], outputs=out)

In [ ]:
tf.keras.backend.clear_session()
model = encode_decode()
model.summary(line_length=118, positions=[.38, .66, .75, 1.], expand_nested=True)

In [ ]:
class MSFCE(Loss):
    
    def call(self, y_true, y_pred):
        bce = BinaryCrossentropy()
        P = tf.reshape(tf.convert_to_tensor(y_pred), [-1])
        T = tf.reshape(tf.cast(y_true, y_pred.dtype), [-1])
        Pmask = tf.math.greater(T, tf.constant(0.5, dtype=tf.float32))
        Nmask = tf.math.less(T, tf.constant(0.5, dtype=tf.float32))
        Vp = tf.boolean_mask(T, Pmask)
        Vp_pred = tf.boolean_mask(P, Pmask)
        Vn = tf.boolean_mask(T, Nmask)
        Vn_pred = tf.boolean_mask(P, Nmask)
        FNCE = bfce(Vp, Vp_pred)
        FPCE = bfce(Vn, Vn_pred)
        return tf.reduce_mean([tf.pow(FPCE, 2), tf.pow(FNCE, 2)])

In [ ]:
class MSFFCE(Loss):
    
    def call(self, y_true, y_pred):
        bfce = BinaryFocalCrossentropy()
        P = tf.reshape(tf.convert_to_tensor(y_pred), [-1])
        T = tf.reshape(tf.cast(y_true, y_pred.dtype), [-1])
        Pmask = tf.math.greater(T, tf.constant(0.5, dtype=tf.float32))
        Nmask = tf.math.less(T, tf.constant(0.5, dtype=tf.float32))
        Vp = tf.boolean_mask(T, Pmask)
        Vp_pred = tf.boolean_mask(P, Pmask)
        Vn = tf.boolean_mask(T, Nmask)
        Vn_pred = tf.boolean_mask(P, Nmask)
        FNCE = bfce(Vp, Vp_pred)
        FPCE = bfce(Vn, Vn_pred)
        return tf.reduce_mean([tf.pow(FPCE, 2), tf.pow(FNCE, 2)])

In [ ]:
class GIoU(Loss):
    
    def call(self, y_true, y_pred):
        #todo
        return

In [ ]:
batch_size = 4
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = MSFFCE()
#BinaryCrossentropy(), BinaryFocalCrossentropy(), MSFCE(), MSFFCE() have been tested to work

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[BinaryIoU()])

In [ ]:
history = model.fit(customGen(batch_size=batch_size), epochs=1,
                    steps_per_epoch=tf.math.ceil(11694 / batch_size))

In [ ]:
plt.plot(history.history['binary_io_u'])
plt.title('model IOU')
plt.ylabel('IOU')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
voxels = Path("models/models-binvox-solid/data")
fvoxels = [f.stem for f in voxels.iterdir() if f.is_file()]
sample = np.random.randint(11695, size=1, dtype=np.int64)[0]
print(sample)
stem = fvoxels[sample]
view0 = f"models/models-screenshots/view0/data/{stem}-00.png"
view1 = f"models/models-screenshots/view1/data/{stem}-01.png"
view2 = f"models/models-screenshots/view2/data/{stem}-02.png"
view3 = f"models/models-screenshots/view3/data/{stem}-03.png"
X0 = np.expand_dims(np.array(Image.open(view0))[:, :, 0:3], axis=0)
X1 = np.expand_dims(np.array(Image.open(view1))[:, :, 0:3], axis=0)
X2 = np.expand_dims(np.array(Image.open(view2))[:, :, 0:3], axis=0)
X3 = np.expand_dims(np.array(Image.open(view3))[:, :, 0:3], axis=0)

vox = f"models/models-binvox-solid/data/{stem}.binvox"
with open(str(vox), 'rb') as f:
    voxel_model = binvox_rw.read_as_3d_array(f)

Y = center(voxel_model.data) * 1
y = (model([X0, X1, X2, X3], training=False).numpy() > 0.5) * 1

fig = plt.figure(figsize=(16, 8))
ax1 = plt.subplot2grid((1, 2), (0, 0), projection='3d')
ax2 = plt.subplot2grid((1, 2), (0, 1), projection='3d')
ax1.voxels(np.squeeze(Y))
ax2.voxels(np.squeeze(y))

plt.show()